# Lab 2: Disaggregated Serving with AIConfigurator

This lab guides you through deploying disaggregated inference (separate prefill and decode workers) using AIConfigurator to find optimal configurations.

## Prerequisites

Before starting, ensure you have:
- ✅ **Completed Lab 1** (namespace, platform, and aggregated serving)
- ✅ Your namespace and Dynamo platform are still running from Lab 1
- ✅ HuggingFace token configured

---

## Configuration

Verify your environment from Lab 1:



In [ ]:
import os

# Verify environment from Lab 1
USER_FRONTEND2_PORT = os.environ.get('USER_FRONTEND2_PORT', '11000')
NAMESPACE = os.environ.get('NAMESPACE', f"dynamo-{os.environ.get('USER', 'unknown')}")
os.environ['RELEASE_VERSION'] = '0.6.0'
os.environ['CACHE_PATH'] = '/data/huggingface-cache'  # Replace with your cache path

print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
print("🎓 Lab 2: Disaggregated Serving Configuration")
print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
print(f"  Namespace:            {NAMESPACE}")
print(f"  Disaggregated Port:   {USER_FRONTEND2_PORT}")
print("")
print("💡 Disaggregated endpoint: localhost:{USER_FRONTEND2_PORT}")
print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")


---

## Part 1: Deploy with AIConfigurator

AIConfigurator helps find optimal configurations for disaggregated serving by analyzing your model and hardware.

### Step 1: Install AIConfigurator



In [ ]:
#In case you were not able to install it earlier.
%%python
# pre-installed in the container
# !pip3 install aiconfigurator


### Step 2: Run Configuration Analysis

Example: Find optimal configuration for Llama 3.1-70B on 16 H200 GPUs with TensorRT-LLM engine.



In [ ]:
!aiconfigurator cli default --model LLAMA3.1_70B --total_gpus 16 --system h200_sxm


### Step 3: Deploy Disaggregated Serving

Use the AIConfigurator output as a reference for vLLM engine and update and deploy `disagg_router.yaml`.

**Note:** In this workshop we use a smaller model and stick to TP=1 for one prefill worker and one decode worker. You can test with `meta-llama/Llama-3.1-70B-Instruct` with TP=2 using four GPUs in total.


In [ ]:
%%bash
# Update disagg_router.yaml
sed -i "s/my-tag/$RELEASE_VERSION/g" disagg_router.yaml
sed -i "s|/YOUR/LOCAL/CACHE/FOLDER|$CACHE_PATH|g" disagg_router.yaml

echo "✓ Configuration updated"
grep "image:" disagg_router.yaml

# Deploy
kubectl apply -f disagg_router.yaml --namespace $NAMESPACE


### Step 4: Monitor Deployment

Wait for all pods (frontend, prefill workers, and decode workers) to be ready:

In [ ]:
%%bash
# Check deployment status
kubectl get dynamographdeployment -n $NAMESPACE

echo ""
echo "Pod status (wait for all pods to be 1/1 Ready):"
kubectl get pods -n $NAMESPACE | grep disagg

# To watch in real-time:
# kubectl get pods -n $NAMESPACE -w

### Step 5: Test Disaggregated Serving

Forward the port and test the endpoint:

In [ ]:
%%bash --bg
# Forward the service port
kubectl port-forward deployment/vllm-v1-disagg-router-frontend $USER_FRONTEND2_PORT:8000 -n $NAMESPACE &

echo "✓ Port forward started on localhost:${USER_FRONTEND2_PORT}"
sleep 5

Test the deployment: Non-streaming response

In [ ]:
!curl localhost:${USER_FRONTEND2_PORT}/v1/chat/completions \
  -H "Content-Type: application/json" \
  -d '{ \
    "model": "Qwen/Qwen2.5-1.5B-Instruct",\
    "messages": [{"role": "user", "content": "Explain the difference between prefill and decode in LLM inference"}], \
    "stream": false,\
    "max_tokens": 100 \
  }'

Test the deployment: streaming response

In [ ]:
!curl localhost:${USER_FRONTEND2_PORT}/v1/chat/completions \
  -H "Content-Type: application/json" \
  -d '{ \
    "model": "Qwen/Qwen2.5-1.5B-Instruct", \
    "messages": [{"role": "user", "content": "Write a short poem about AI"}], \
    "stream": true, \
    "max_tokens": 100 \
  }'

---

## Part 2: Understanding Disaggregated Architecture

### Key Differences from Aggregated Serving

| Aspect | Aggregated (Lab 1) | Disaggregated (Lab 2) |
|--------|-------------------|---------------------|
| **Workers** | Single worker type handles both prefill and decode | Separate prefill and decode workers |
| **Routing** | Round-robin across workers | KV-aware routing between prefill and decode |
| **Use Case** | Single-node models, simpler topology | Multi-node models, optimized resource usage |
| **Efficiency** | Good for small-medium models | Better for large models with different compute profiles |

### When to Use Disaggregated Serving

- **Large models** that require multiple GPUs or tensor parallelism
- **Workloads with different prefill/decode characteristics**
- **Optimizing resource allocation** (different GPU types for prefill vs decode)
- **Scaling prefill and decode independently**

---

## Part 3: Cleanup

Delete the disaggregated deployment when done:

In [ ]:
%%bash
# Delete disaggregated deployment
kubectl delete dynamographdeployment vllm-v1-disagg-router -n $NAMESPACE

echo "✓ Disaggregated deployment deleted"
echo ""
echo "Note: Your namespace and platform from Lab 1 are still running"

---

## Troubleshooting

### Check Disaggregated Deployment Status

In [ ]:
%%bash
# Check all disaggregated pods
kubectl get pods -n $NAMESPACE | grep disagg

# Check prefill worker logs
PREFILL_POD=$(kubectl get pods -n $NAMESPACE | grep prefill | head -1 | awk '{print $1}')
if [ -n "$PREFILL_POD" ]; then
    echo "Prefill worker logs:"
    kubectl logs $PREFILL_POD -n $NAMESPACE --tail=30
fi

# Check decode worker logs
DECODE_POD=$(kubectl get pods -n $NAMESPACE | grep decode | head -1 | awk '{print $1}')
if [ -n "$DECODE_POD" ]; then
    echo "Decode worker logs:"
    kubectl logs $DECODE_POD -n $NAMESPACE --tail=30
fi

### Common Issues

1. **Pods stuck in Pending**: Check if sufficient GPU resources are available. Disaggregated serving requires more GPUs than aggregated.
   ```bash
   kubectl describe pod <pod-name> -n $NAMESPACE | grep -A 5 Events
   ```

2. **Communication issues between prefill and decode**: Check NATS connectivity and ensure all workers registered correctly.
   ```bash
   kubectl logs -l component=Frontend -n $NAMESPACE --tail=50
   ```

3. **Model loading failures**: Check that the model is compatible with disaggregated serving and tensor parallelism settings are correct.

---

## Summary

### What You Learned
- ✅ How to use AIConfigurator to find optimal configurations
- ✅ Deploying disaggregated serving (separate prefill/decode workers)
- ✅ Understanding differences between aggregated and disaggregated architectures
- ✅ When to use disaggregated serving for large models

### Key Takeaways
- **AIConfigurator** automates finding optimal tensor parallelism and resource allocation
- **Disaggregated serving** separates prefill and decode for better resource utilization
- **KV-aware routing** is essential for coordinating between prefill and decode workers
- **Scaling** prefill and decode independently can optimize for different workload patterns

---

## Additional Resources

- 📖 [Dynamo Documentation](https://docs.dynamo.nvidia.com)
- 🔧 [AIConfigurator Guide](../../dynamo/docs/tools/aiconfigurator.md)
- 🏗️ [Disaggregated Serving Architecture](../../dynamo/docs/architecture/disaggregated.md)
- 📊 [AIPerf Benchmarking Tool](https://github.com/ai-dynamo/aiperf)

---

**Congratulations! 🎉** You've successfully deployed disaggregated inference with Dynamo!
